# BGT zadanie 6
## Autors
- Paweł Benkowski
- Stanisław Lutkiewicz"
### Task
Read movies and their ratings.  
Get recommendation for particular user.  


In [1]:
import sys
!{sys.executable} -m pip install requests


In [2]:
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession, functions
from pyspark.ml.recommendation import ALS
from pathlib import Path
import csv
import os
import requests

In [3]:
sparkSession = SparkSession.builder.master("local").appName("BGT 6").getOrCreate()

In [16]:
'''
Read and format data for further calculation
'''
fileName = "BigData - Filmy.csv"
userRatings = {}
titles = {}
combinedUsersTitlesRating = []
numberToTitle = {}
numberToUser = {}
with open(fileName) as csvFile:
    csvReader = csv.reader(csvFile,delimiter=";")
    for row in csvReader:
        # print(row)
        user = row[0]
        filmy = row[1::2]
        oceny = row[2::2]
        for film in set(filmy):
            if film not in titles.keys():
                titles[film]=len(titles)    #encode titles
        userRatings[user] = dict(zip(filmy, oceny))
users = dict(zip(userRatings.keys(),list(range(len(userRatings.keys())))))

# combine movies and users
for name,movies in userRatings.items():
    numberToUser[users[name]] = name # for later decoding
    for title,rating in movies.items():
        numberToTitle[titles[title]] = title # for later decoding
        combination = (users[name],titles[title],float(rating))
        combinedUsersTitlesRating.append(combination)

print(numberToUser)

{0: 'Paweł Czapiewski', 1: 'Ola Piętka', 2: 'Robert Deyk', 3: 'Jaromir Daruk', 4: 'Benedykt Kościński', 5: 'Paweł Benkowski', 6: 'Filip Mikołajek', 7: 'Jakub Kulaszewicz', 8: 'Robert Mielewczyk', 9: 'Stanisław Lutkiewicz', 10: 'Wiktor Maj'}


In [17]:
'''
https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.recommendation
create model adn get recommendation
'''
df = sparkSession.createDataFrame(combinedUsersTitlesRating, ["user", "item", "rating"])


In [6]:
als = ALS(rank=10, seed=0)
als.setMaxIter(5)
als.getMaxIter()
als.setRegParam(0.1)
als.getRegParam()
als.clear(als.regParam)
model = als.fit(df)
model.getBlockSize()

4096

In [13]:
userId = 0

userTitles = [x[1] for x in combinedUsersTitlesRating if x[0] == userId]
titlesWithoutSelectedUser = [(userId,x) for x in titles.values() if x not in userTitles]

titlesWithoutSelectedUserDf = sparkSession.createDataFrame(titlesWithoutSelectedUser, ["user", "item"])
predictions = sorted(model.transform(titlesWithoutSelectedUserDf).collect(), key=lambda r: r[2],reverse=True)


In [15]:
# print predictions
counter = 0
for row in predictions[:5]:
    print(numberToTitle[row[1]])


Interstellar
Marsjanin
Player One
Mindhunter
Chernobyl
